In [1]:
import numpy as np

# pydrostat
Muscular hydrostat simulation using constrained dynamics

This notebook demonstrates a simulation of a muscular hydrostat arm. 



# 1) Define a structure to simulate

The structure is the arm or tentacle or cell being simulated. It is any set of vertices connected by edges, faces, and cells (although it's not required to have any of those connections). This notebook will demonstrate with an arm made of cubic cells. 

In [2]:
from pydrostat.structure import structures

arm_height = 1
arm_builder = structures.CubicArmBuilder(arm_height)

# 2) Define an environment for the structure

An environment has food and obstacles. The food emmits a scent which diffuses through the environment around the obstacles (ie obstacles block the scent). This requires a simulation of the diffusion of the scent which approaches some steady state distribution of scent concentration.

Add this environment to the arm builder.

In [3]:
from pydrostat.environment import environment
from pydrostat.environment.obstacles import convex_polytope
from pydrostat.environment import food

food_loc = np.array([[-4, 0, 3]])
food_mag = 1
food_instance = food.Food(food_loc, food_mag)

rect_obst = convex_polytope.build_rectangular_obstacle([-2.5, -1, -1], [-1.5 ,9, 9])

env = environment.Environment(dim = 3, obstacles = [rect_obst], foods = [food_instance])
arm_builder.add_environment(env)


Loop: 50, Error: 1.000
Loop: 100, Error: 1.000
Loop: 150, Error: 0.296
Loop: 200, Error: 0.148
Loop: 250, Error: 0.090
Loop: 300, Error: 0.060
Converged!


# 3) Define sensors and a controller for the structure

A structure takes sensory measurements of its environment and passes the sense data to a controller. Each type of controller may require a specific type of sensor data, or some combination of sensor data. 

In [4]:
from pydrostat.sensing.vertex_chemoceptors import VertexChemoceptors
from pydrostat.control.hand_tuned_gradient import HandTunedGradient

arm_builder.add_sensor(VertexChemoceptors())
# arm_builder.add_controller(HandTunedGradient())


# 4) Add constraints

Each constraint defines some sort of equality or inequality constraint that must be maintained throughout the simulation. This can be something like constant volume of a cell, minimum or maximum edge length, etc. 

In [5]:
from pydrostat.constraint.constant_volume import ConstantVolume
from pydrostat.constraint.fixed_vertex import FixedVertex

arm_builder.add_constraint(ConstantVolume())
arm_builder.add_constraint(FixedVertex([0,1,2,3]))

# 5) Build the arm

In [6]:
arm = arm_builder.construct_arm()

In [ ]:

initial_positions = arm.positions
arm.apply_external_forces(4, np.array([0,0,1]))

In [8]:
# arm.iterate(0.01)
# print(arm.positions - initial_positions)

In [9]:
from pydrostat.render.display import DisplayStructure

render_display = DisplayStructure(env, [arm])
render_display.main_loop()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

/home/alex/code/muscular_hydrostat/venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
